--- 18/08/2025 kotla sai charan

In [ ]:
!pip install --no-deps -U transformers==4.52.4
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps -U bitsandbytes accelerate xformers==0.0.29.post3 peft==0.15.2 trl==0.18.2 cut_cross_entropy 
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps 'unsloth==2025.6.12' 'unsloth_zoo==2025.6.8'
!pip install 'triton==3.2.0' safetensors regex tokenizers
!pip install evaluate nltk bert_score rouge_score spacy
!pip install --pre --upgrade --no-cache-dir torch torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cu128

In [1]:
!git clone https://github.com/GAD-cell/vlm-grpo.git
%cd vlm-grpo
!pip install -e .

Cloning into 'vlm-grpo'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 246 (delta 46), reused 48 (delta 30), pack-reused 174 (from 1)
Receiving objects: 100% (246/246), 787.19 KiB | 3.17 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/charan_work/vlm-grpo
Obtaining file:///charan_work/vlm-grpo
  Preparing metadata (setup.py) ... done
  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached cut_cross_entropy-25.1.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached hf_transfer-0.1.9-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.me

you must run this first

In [2]:
import vlmgrpo

/charan_work/vlm-grpo/vlmgrpo/patches/unsloth_patch.py:100: SyntaxWarning: invalid escape sequence '\.'
  name = re.sub("\.([\d]{1,})\.", r"[\1].", name)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth patched for VLMs GRPO training


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0.dev20250723+cu128)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import peft
peft.__version__

'0.15.2'

In [ ]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit",
    # 'Cherran/VL_lr_5e5_medical_qwen2point5_7b',
    max_seq_length = 2048,
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2025.6.12: Fast Qwen2_5_Vl patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 5070 Ti. Num GPUs = 1. Max memory: 15.479 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0.dev20250723+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.97G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [ ]:
# model = FastVisionModel.get_peft_model(
#     model,
#     finetune_vision_layers     = True, # False if not finetuning vision layers
#     finetune_language_layers   = True, # False if not finetuning language layers
#     finetune_attention_modules = True, # False if not finetuning attention layers
#     finetune_mlp_modules       = True, # False if not finetuning MLP layers

#     r = 32,
#     lora_alpha = 32,
#     lora_dropout = 0,
#     bias = "none",
#     random_state = 3407,
#     use_rslora = False,
#     loftq_config = None, # And LoftQ
#     # target_modules = "all-linear", # Optional now! Can specify a list if needed
# )

In [8]:
from datasets import load_dataset
dataset = load_dataset('Cherran/train_extra_rewards_GRPO_medical_summaries', split = 'train')
dataset

README.md:   0%|          | 0.00/571 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/82.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/735 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'post', 'image_desc', 'prompt', 'summary', 'drug_names', 'non_physical_adverse_effects', 'physical_adverse_effects'],
    num_rows: 735
})

In [6]:
import trl
trl.__version__

'0.18.2'

This code is not working for multiple images, so removed all images with len>1

In [9]:
dataset = dataset.filter(lambda sample: len(sample['image']) <= 1)
dataset

Filter:   0%|          | 0/735 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'post', 'image_desc', 'prompt', 'summary', 'drug_names', 'non_physical_adverse_effects', 'physical_adverse_effects'],
    num_rows: 731
})

In [ ]:
def convert_to_conversation(sample):

    resized_images = []
    for image_object in sample["image"]:
        resized_images.append(image_object.resize((512, 512)))

    user_content = [
        {"type": "text", "text": sample['prompt']}
    ]

    for resized_image_object in resized_images:
        user_content.append(
            {"type": "image", "image": resized_image_object}
        )

    conversation = [
        {
            "role": "user",
            "content": user_content
        }
    ]

    return {
        "prompt": conversation,
        "image": resized_images,
        "image_desc": sample['image_desc'],
        "answer": sample['summary'],
        "drug_names" : sample['drug_names'],
        "physical_adverse_effects" : sample['physical_adverse_effects'],
         "non_pdv" : sample['non_physical_adverse_effects']
    }

#### Adding dummy image is important, this repo https://github.com/GAD-cell/vlm-grpo.git is not supporting as of today 18/08/2025 

In [ ]:
from PIL import Image

dummy_image = Image.new('RGB', (512, 512), color = 'black')

def add_dummy_image(example):
    if not example["image"]:
        example["image"] = [dummy_image]
    return example

dataset = dataset.map(add_dummy_image)
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

Map:   0%|          | 0/731 [00:00<?, ? examples/s]

In [12]:
converted_dataset[2]

{'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'You are an excellent medical paragraph generator. I have drug names and Adverse Drug reactions Keywords. Consolidate all information in one paragrah without adding any extra knowledge or text.\n  drug_names: Taxol\n  physical_adverse_effects: Pain in left hand/thumb, port protruding through skin, brace on hand\n\n  Strcitly output the paragraph no explanations.\n'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=512x512>}]}],
 'image': [<PIL.Image.Image image mode=RGB size=512x512>],
 'image_desc': '',
 'answer': 'Taxol has been associated with adverse effects including pain in the left hand/thumb, a port protruding through the skin, and the necessity of a brace on the hand.',
 'drug_names': 'Taxol',
 'physical_adverse_effects': 'Pain in left hand/thumb, port protruding through skin, brace on hand',
 'non_pdv': 'Feeling of frustration or hopelessness ("can\'t catch a break")'}

In [ ]:
dataset_size_bytes = dataset.dataset_size

print(f"The disk size of 'converted_dataset' is {dataset_size_bytes/1024/1024} bytes.")

The disk size of 'converted_dataset' is 50.94754409790039 bytes.


In [13]:
import evaluate

rouge = evaluate.load("rouge")

def compute_rouge_l(prediction: str, reference: str) -> float:
    result = rouge.compute(predictions=[prediction], references=[reference], rouge_types=["rougeL"])
    return result["rougeL"]


In [14]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
def compute_bleu(prediction: str, reference: str) -> float:
    ref_tokens = [reference.split()]  # BLEU expects list of reference sentences
    pred_tokens = prediction.split()
    smoothing = SmoothingFunction().method1
    return sentence_bleu(ref_tokens, pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothing)


In [15]:
def compute_reward(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    rewards = []
    for res, ans in zip(responses, answer):
      if res == ans :
          reward = 1.2
          rewards.append(reward)
          continue

      reward = 0.0
      rouge_l = compute_rouge_l(res, ans)
      bleu = compute_bleu(res, ans)
      reward += (rouge_l)*0.6 + bleu*0.4

      rewards.append(reward)

    print(f"len of rewards : f{len(rewards)} and avg reward {sum(rewards)/len(rewards)}")
    return rewards

In [18]:
def length_reward(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    rewards = []

    avg_answer_len = np.mean([len(ans.split()) for ans in answer])
    sigma = avg_answer_len * 0.25

    for res, ans in zip(responses, answer):
        len_res = len(res.split())
        len_ans = len(ans.split())

        len_diff = abs(len_res - len_ans)

        # Gaussian reward function: e^(-(x^2) / (2 * sigma^2))
        # This gives a reward of 1.0 for a perfect match, smoothly decreasing to 0.
        reward = np.exp(-0.5 * (len_diff / sigma) ** 2)

        rewards.append(reward)

    return rewards

In [19]:
import re

def drug_adverse_preservation_reward(prompts, completions, answer, drug_names, physical_adverse_effects, non_pdv, **kwargs) -> list[float]:
    """
    Calculates a total reward for each response by summing two separate binary rewards:
    1. A reward for correctly listing drugs for that specific response.
    2. A reward for correctly listing adverse effects for that specific response.
    The maximum possible reward for each response is 2.0.
    """
    responses = [completion[0]['content'] for completion in completions]
    rewards = []

    # Zip all corresponding lists together to process each response with its unique ground truth.
    for res, pmpt, drugs_str, physical_effects_str, non_pdv_str in zip(
        responses, prompts, drug_names, physical_adverse_effects, non_pdv
    ):
        res_lower = res.lower()

        drug_set = {d.strip().lower() for d in drugs_str.split(',') if d.strip()}

        all_drugs_found = all(drug in res_lower for drug in drug_set)
        drug_reward = 0.5 if all_drugs_found else 0.0

        is_physical_prompt = 'physical_adverse_effects:' in pmpt

        adverse_effects_str = physical_effects_str if is_physical_prompt else non_pdv_str

        adverse_set = {a.strip().lower() for a in adverse_effects_str.split(',') if a.strip()}

        all_adverse_effects_found = all(effect in res_lower for effect in adverse_set)
        adverse_reward = 0.5 if all_adverse_effects_found else 0.0

        total_reward = drug_reward + adverse_reward
        # print(total_reward)
        rewards.append(total_reward)

    return rewards

In [20]:
import torch
import torch.nn.functional as F
from transformers import AutoProcessor, AutoModel
import spacy
from PIL import Image
from spacy.lang.en.stop_words import STOP_WORDS

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

NLP_MODEL = spacy.load("en_core_web_sm")

CLIP_PROCESSOR = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
CLIP_MODEL = AutoModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)

print(f"Models loaded and using device: {DEVICE}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Models loaded and using device: cuda


In [21]:
import torch
import torch.nn.functional as F
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

CLIP_DEVICE = DEVICE
# CLIP_MODEL.to(CLIP_DEVICE)

def calculate_visual_grounding_reward(
    completions: list, 
    image: list, 
    image_desc: list, 
    **kwargs  
) -> list[float]:
    """
    Calculates a reward based on how well the completion is grounded in the image content.
    """
    rewards = []
    
    # Safely extract responses
    responses = [
        comp[0].get('content', '') if comp and isinstance(comp, list) and comp[0] else '' 
        for comp in completions
    ]

    for response, img_item, img_desc in zip(responses, image, image_desc):
        # --- 1. Input Validation ---
        if not response or not img_desc:
            rewards.append(0.0)
            continue
            
        try:
            if not img_item:
                logging.warning("Empty image list encountered. Skipping.")
                rewards.append(0.0)
                continue
            img_input = img_item[0]
            
            
            if img_input.getbbox() is None:
                logging.warning("Image is blank or all black. Skipping.")
                rewards.append(0.0)
                continue
        except (FileNotFoundError, IndexError) as e:
            logging.error(f"Could not process image: {e}. Skipping.")
            rewards.append(0.0)
            continue

        # --- 2. Extract Key Phrases from Description ---
        doc = NLP_MODEL(img_desc.lower())
        all_chunks = list({chunk.text.strip() for chunk in doc.noun_chunks if chunk.text.lower() not in STOP_WORDS})

        if not all_chunks:
            logging.info(f"No usable noun chunks found in description: '{img_desc}'.")
            rewards.append(0.0)
            continue

        # --- 3. Score Phrases against Image using CLIP ---
        try:
            # Performance: Move inputs directly to the model's device
            inputs = CLIP_PROCESSOR(
                text=all_chunks,
                images=img_input,
                return_tensors="pt",
                padding=True
            ).to(CLIP_DEVICE)

            with torch.no_grad():
                outputs = CLIP_MODEL(**inputs)
            
            # Get scores and find top 3 visually grounded chunks
            scores = outputs.logits_per_image.squeeze()
            chunk_scores = sorted(zip(all_chunks, scores), key=lambda x: x[1], reverse=True)
            top_chunks_with_scores = chunk_scores[:2]
            
            top_chunks = [item[0] for item in top_chunks_with_scores]
            top_scores = torch.tensor([item[1] for item in top_chunks_with_scores])
            
        except Exception as e:
            logging.error(f"Error during CLIP processing for '{img_desc}': {e}")
            rewards.append(0.0)
            continue

        # --- 4. Calculate Final Weighted Reward ---
        weights = F.softmax(top_scores, dim=0)
        final_reward = 0.0
        response_lower = response.lower()
        print(top_chunks)
        for i, chunk in enumerate(top_chunks):
            if chunk in response_lower:
                final_reward += weights[i].item()
        
        rewards.append(final_reward)

    return rewards

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, torch.nn.functional as F

tok = AutoTokenizer.from_pretrained("manueldeprada/FactCC")
clf = AutoModelForSequenceClassification.from_pretrained("manueldeprada/FactCC").eval()

In [ ]:

def factual_consistency_reward(
    source_texts: list[str],
    completions: list[str],
    rescale_floor: float = 0.875,
    sensitivity: float = 3.0,
    **kwargs
) -> list[float]:
    rewards = []
    responses = [completion[0]['content'] for completion in completions]

    for src, hyp in zip(source_texts, responses):
        inputs = tok(src, hyp, truncation=True, padding=True, return_tensors="pt")

        with torch.no_grad():
            logits = clf(**inputs).logits.squeeze()
            probs = F.softmax(logits, dim=-1)
            prob_consistent = probs[1].item()

            if prob_consistent < rescale_floor:
                rescaled_prob = 0.0
            else:
                rescaled_prob = (prob_consistent - rescale_floor) / (1.0 - rescale_floor)
            reward = rescaled_prob ** sensitivity
            rewards.append(reward)

    return rewards


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def domain_specific_hallucination_penalty(
    prompts: list[str],
    completions: list[str],
    answer: list[str],
    drug_names: list[str],
    image_desc: list[str],
    physical_adverse_effects: list[str],
    non_pdv: list[str],
    **kwargs
) -> list[float]:
    """
    Penalizes the model for generating noun phrases not found in the source
    context or a pre-defined list of domain-specific terms.
    """
    
    responses = [
        comp[0].get('content', '') if comp and isinstance(comp, list) and comp[0] else '' 
        for comp in completions
    ]
    
    base_texts = [(ans + " " + img_des).lower() for ans, img_des in zip(answer, image_desc)]
    response_texts = [res.lower() for res in responses]

    # --- 2. Efficient Batch Processing with nlp.pipe ---
    
    # Process all source and response texts at once for a major speedup.
    base_docs = list(nlp.pipe(base_texts))
    response_docs = list(nlp.pipe(response_texts))

    # --- 3. Main Loop ---
    
    rewards = []
    iterator = zip(
        base_docs, response_docs, drug_names, physical_adverse_effects, non_pdv
    )

    for base_doc, res_doc, drugs_str, effects_str, non_effects_str in iterator:
        
        # --- 4. Build Whitelist of Allowed Chunks ---
        
        # Extract noun chunks from the pre-processed base document
        allowed_chunks = {
            chunk.text.strip() for chunk in base_doc.noun_chunks 
            if chunk.text not in STOP_WORDS
        }

        # Add known domain-specific terms to the whitelist
        known_drugs = {drug.strip().lower() for drug in drugs_str.split(',') if drug.strip()}
        known_effects = {effect.strip().lower() for effect in effects_str.split(',') if effect.strip()}
        known_non_phy = {non_effect.strip().lower() for non_effect in non_effects_str.split(',') if non_effect.strip()}
        
        allowed_chunks.update(known_drugs, known_effects, known_non_phy)

        # --- 5. Identify Hallucinations and Calculate Penalty ---
        
        # Extract noun chunks from the pre-processed response document
        response_chunks = {
            chunk.text.strip() for chunk in res_doc.noun_chunks 
            if chunk.text not in STOP_WORDS
        }

        hallucinated_chunks = response_chunks - allowed_chunks
        
        # Calculate a penalty score, capped at 1.0
        penalty = min(1.0, 0.25 * len(hallucinated_chunks))
        
        rewards.append(-penalty*0.25)

    return rewards

In [ ]:
# torch._dynamo.config.accumulated_recompile_limit = 64 #use this fot recompile error or a google search would resolve issue

AttributeError: torch._dynamo.config.recompile_limit does not exist

In [3]:
import unsloth_zoo
unsloth_zoo.__version__

'2025.6.8'

In [16]:
from unsloth import is_bf16_supported
from vlmgrpo import VLMGRPOTrainer
from trl import GRPOConfig

training_args = GRPOConfig(
    learning_rate = 2e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bf16_supported(),
    fp16 = not is_bf16_supported(),
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory,

    # max_prompt_length = 2048,
    max_completion_length = 128,
    num_train_epochs = 1, # Set to 1 for a full training run
    # max_steps = 70,
    save_steps = 15,
    max_grad_norm = 1.0,
    report_to = "none", #"wandb", # Can use Weights & Biases
    output_dir = "outputs_grpo",
    # beta = 0.0
)

trainer = VLMGRPOTrainer(
    model = model,
    processing_class=tokenizer, # MUST put unsloth processor here!
    reward_processing_classes = tokenizer, #Here also
    reward_funcs = [
        compute_reward,
        # length_reward,
        # drug_adverse_preservation_reward,
        # calculate_visual_grounding_reward,
        # domain_specific_hallucination_penalty,
    ],
    args = training_args,
    train_dataset = converted_dataset,
    grad_verbose=True
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 731 | Num Epochs = 1 | Total steps = 183
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 103,043,072 of 7,000,000,000 (1.47% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 128000, 'temperature': 1e-06, 'repetition_penalty': 1.05, 'bos_token_id': 151643}. If this is not desired, please set these values explicitly.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


len of rewards : f16 and avg reward 0.5523432785141421


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


[DEBUG] Loss: -0.05372237414121628
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.1219
Unsloth: Will smartly offload gradients to save VRAM!
[DEBUG] Loss: -0.05372237786650658
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.2438
[DEBUG] Loss: 0.10830484330654144
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.3137
[DEBUG] Loss: 0.00022319781419355422
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.3137


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / compute_reward / mean,rewards / compute_reward / std
1,0.001100,0.552343,0.030738,30.375000,25.000000,42.000000,0.000000,30.375000,25.000000,42.000000,1.061415,0.552343,0.198418
2,0.000800,0.495064,0.011618,37.625000,32.000000,48.000000,0.000000,37.625000,32.000000,48.000000,0.845102,0.495064,0.194083
3,0.000500,0.566156,0.031221,46.250000,29.000000,82.000000,0.000000,46.250000,29.000000,82.000000,0.514093,0.566156,0.166885
4,0.000800,0.669620,0.011657,37.062500,26.000000,61.000000,0.000000,37.062500,26.000000,61.000000,0.843835,0.669620,0.142635
5,0.000900,0.353183,0.000000,45.250000,31.000000,59.000000,0.000000,45.250000,31.000000,59.000000,0.763771,0.353183,0.236487
6,0.000200,0.463427,0.032214,84.875000,73.000000,101.000000,0.000000,84.875000,73.000000,101.000000,0.167487,0.463427,0.054553
7,0.000300,0.832248,0.011880,31.000000,20.000000,40.000000,0.000000,31.000000,20.000000,40.000000,0.307679,0.832248,0.054435
8,0.000200,0.495051,0.020027,50.312500,17.000000,128.000000,0.062500,45.133335,17.000000,121.000000,0.296973,0.495051,0.262431
9,0.000600,0.727647,0.000000,32.500000,30.000000,35.000000,0.000000,32.500000,30.000000,35.000000,0.543326,0.727647,0.049539
10,0.000400,0.703201,0.000000,26.250000,21.000000,35.000000,0.000000,26.250000,21.000000,35.000000,0.323643,0.703201,0.128671


...

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


len of rewards : f16 and avg reward 0.6459186919980757
[DEBUG] Loss: 0.0005498395767062902
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0002
[DEBUG] Loss: 0.0004651999333873391
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0004
[DEBUG] Loss: 0.00048797245835885406
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0005
[DEBUG] Loss: 0.0003577876486815512
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0006


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


len of rewards : f16 and avg reward 0.6515328859276625
[DEBUG] Loss: 0.0001532670867163688
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0001
[DEBUG] Loss: 0.00037513207644224167
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0004
[DEBUG] Loss: 0.0003773758071474731
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0010
[DEBUG] Loss: 0.0003092377446591854
[DEBUG] Params with grad: 392 / 1441
[DEBUG] Grad norm: 0.0014


TrainOutput(global_step=182, training_loss=0.0010957690152521295, metrics={'train_runtime': 3225.4564, 'train_samples_per_second': 0.227, 'train_steps_per_second': 0.057, 'total_flos': 0.0, 'train_loss': 0.0010957690152521295})

In [27]:
%cd vlm-grpo

/charan_work/vlm-grpo


In [ ]:
from huggingface_hub import HfApi

model_path = "outputs_grpo/checkpoint-180"
username = "Cherran"
MODEL_NAME = "grpo_only_compute_later_l2e5_VL_medical_qwen2point5_7b"
api = HfApi(token=  "HF_API")

api.create_repo(
    repo_id = f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    repo_id = f"{username}/{MODEL_NAME}",
    folder_path = model_path
)

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

rng_state.pth:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/164M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.80k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Cherran/grpo_only_compute_later_l2e5_VL_medical_qwen2point5_7b/commit/ee41faf0e1b731e7e72cc9726dd1e7646f9638a7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='ee41faf0e1b731e7e72cc9726dd1e7646f9638a7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Cherran/grpo_only_compute_later_l2e5_VL_medical_qwen2point5_7b', endpoint='https://huggingface.co', repo_type='model', repo_id='Cherran/grpo_only_compute_later_l2e5_VL_medical_qwen2point5_7b'), pr_revision=None, pr_num=None)

In [20]:
%cd ..

/charan_work


In [22]:
import pandas as pd
df_val = pd.read_csv('val_VL_medical_qwen_draft_5rewards_grpo_only_drugADV.csv')
df_val.head(1)

,Unnamed: 0,Links,Posts,Preprocessed Posts,drug_names,adverse_effects,severity,physical_adverse_effects,non_physical_adverse_effects,side_harmful_effects,...,phy_summary,non_phy_summary,VL_qwen_phy_sum_5rew_visualgr_grpo_lr2e5,VL_qwen_non_phy_sum_5rew_visualgr_grpo_lr2e5,VL_qwen_phy_sum_4rew_visualgr_grpo_lr2e5,VL_qwen_non_phy_sum_4rew_visualgr_grpo_lr2e5,VL_qwen_phy_sum_grpo_3rew_lr2e5,VL_qwen_non_phy_sum_3rew_grpo_lr2e5,VL_qwen_phy_sum_only_drugADV_grpo_lr2e5,VL_qwen_non_phy_sum_only_drugADV_grpo_lr2e5
0,0,https://csn.cancer.org/discussion/comment/1239...,Xeloda on the Web.. Awesome website\n I switc...,xeloda on the web awesome website i switched o...,Xeloda,Yes,Mild,HandFoot syndrome,NaN,NaN,...,Xeloda is associated with adverse drug reactio...,NaN,Xeloda is associated with adverse effects such...,NaN,A patient receiving Xeloda treatment has devel...,NaN,The image shows a patient experiencing physica...,NaN,Xeloda is associated with adverse effects incl...,NaN


In [23]:
from datasets import load_dataset

dataset = load_dataset("Cherran/GRPO_actualvalid_legit_medical_summaries", split = 'train')
dataset

README.md:   0%|          | 0.00/594 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/29.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'phy_prompt', 'non_phy_prompt', 'phy_summary', 'non_phy_summary', 'drug_names', 'non_physical_adverse_effects', 'physical_adverse_effects'],
    num_rows: 300
})

you can skip this step, no need to add dummy images

In [24]:
from PIL import Image

dummy_image = Image.new('RGB', (512, 512), color = 'black')

def add_dummy_image(example):
    if not example["image"]:
        example["image"] = [dummy_image]
    return example

dataset = dataset.map(add_dummy_image)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
from PIL import Image

def generate_response(sample, prompt_type='phy_prompt', temperature=1.0):

    # --- Part 1: Construct the conversation ---

    user_content = [
        {"type": "text", "text": sample[prompt_type]}
    ]

    images = sample.get("image", [])
    if images and not isinstance(images, list):
        images = [images]

    # --- New: Image Resizing Logic ---
    processed_images = []
    if images:
        for img_data in images:
            # Open the image whether it's a path (str) or already a PIL Image
            img = Image.open(img_data) if isinstance(img_data, str) else img_data

            # Resize the image and append to the new list
            resized_img = img.resize((512, 512))
            processed_images.append(resized_img)

    for _ in processed_images:
        user_content.append({"type": "image"})

    conversation = [
        {"role": "user", "content": user_content}
    ]

    final_prompt_string = tokenizer.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=True
    )

    # --- Part 2: Conditionally tokenize inputs ---

    # Use the new 'processed_images' list
    if processed_images:
        print("Processing with text and resized images (512x512)...")
        inputs = tokenizer(
            text=final_prompt_string,
            images=processed_images, # Pass the resized images
            return_tensors="pt"
        ).to("cuda")
    else:
        print("Processing with text only...")
        inputs = tokenizer(
            text=final_prompt_string,
            return_tensors="pt"
        ).to("cuda")

    # --- Part 3: Generate and decode response ---

    output_ids = model.generate(**inputs, max_new_tokens=128, use_cache=True, temperature=temperature, min_p=0.1)

    input_ids_length = inputs["input_ids"].shape[1]
    newly_generated_ids = output_ids[:, input_ids_length:]

    response_text = tokenizer.batch_decode(newly_generated_ids, skip_special_tokens=True)[0]

    return response_text.strip()



In [36]:
dataset[45]

{'image': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512>],
 'phy_prompt': 'You are an excellent medical paragraph generator. I have drug names and Adverse Drug reactions Keywords. Consolidate all information in one paragrah without adding any extra knowledge or text.\n  drug_names: ipilumumab, nivomulab, antihistamines, Imodium, oral steroids\n  physical_adverse_effects: itchy rash, skin crawling sensation, diarrhoea, loss of appetite, weight loss (lost a stone and a half), immunotherapy induced colitis\n\n  Strcitly output the paragraph no explanations.\n',
 'non_phy_prompt': '',
 'phy_summary': 'Ipilimumab and nivolumab, along with antihistamines, Imodium, and oral steroids, may lead to a range of adverse physical effects including itchy rash, skin crawling sensation, diarrhea, loss of appetite, significant weight loss (up to a stone and a half), and immunotherapy-induced colitis.',
 'non_phy_summary': 'nan',
 'drug_names': 'ipilumumab, nivomulab, antihistamines, Imod

In [ ]:
from tqdm import tqdm

df_val['VL_qwen_phy_sum_only_rougeBl_grpo_lr2e5'] = ''

for index in tqdm(range(0, len(dataset))):
    sample = dataset[index]
    if sample['phy_prompt'] == '' :
        continue
    response = generate_response(sample, prompt_type = 'phy_prompt', temperature = 1.0)
    print(response)
    df_val.at[index , 'VL_qwen_phy_sum_only_rougeBl_grpo_lr2e5'] = response

  0%|          | 0/300 [00:00<?, ?it/s]

Processing with text and resized images (512x512)...


  0%|          | 1/300 [00:11<59:15, 11.89s/it]

Xeloda is associated with physical adverse effects, including Hand-Foot syndrome, characterized by redness and swelling on the palms of the hands and soles of the feet. The image shows a close-up of a hand and feet affected by these symptoms.
Processing with text and resized images (512x512)...


  1%|          | 2/300 [00:13<29:12,  5.88s/it]

...

Taxol is associated with physical adverse effects including leg pain, numbness in fingers, and numbness in toes.
Processing with text and resized images (512x512)...


100%|█████████▉| 299/300 [05:23<00:00,  1.13it/s]

Co-codamol is associated with physical adverse effects, including discomfort in the lower abdomen.
Processing with text and resized images (512x512)...


100%|██████████| 300/300 [05:24<00:00,  1.08s/it]

Cape and morphine are associated with various physical adverse effects, including pain in the scar area, fatigue, dry mouth, indigestion, constipation, muscle weakness in the legs, and dizziness.


In [ ]:
df_val['VL_qwen_non_phy_sum_only_rougeBl_grpo_lr2e5'] = ''
for index in tqdm(range(0, len(dataset))):
    sample = dataset[index]
    if sample['non_phy_prompt'] == '' :
        continue
    response = generate_response(sample, prompt_type = 'non_phy_prompt', temperature = 1.0)
    print(response)
    df_val.at[index , 'VL_qwen_non_phy_sum_only_rougeBl_grpo_lr2e5'] = response

  0%|          | 0/300 [00:00<?, ?it/s]

Processing with text and resized images (512x512)...


  1%|          | 3/300 [00:02<03:29,  1.42it/s]

Tamoxifen and aromatase inhibitors, such as letrozole and anastrozole, are commonly used for hormone therapy in breast cancer treatment. Non-physical adverse effects reported include anxiety, which some patients experience due to concerns about the potential link between these medications and the risk of uterine cancer.
Processing with text and resized images (512x512)...


  2%|▏         | 6/300 [00:03<02:21,  2.07it/s]

...

Letrozole, Arimidex, Exemestane, Tamoxifen, and Femara are associated with non-physical adverse effects including insomnia.
Processing with text and resized images (512x512)...


 99%|█████████▉| 297/300 [02:06<00:00,  3.46it/s]

Carb and Taxol are associated with non-physical adverse effects including feeling self-conscious, feeling very tired, and feeling exhausted.
Processing with text and resized images (512x512)...


100%|██████████| 300/300 [02:07<00:00,  2.35it/s]

Cape and morphine are associated with non-physical adverse effects, including mental confusion where patients may feel not completely with it.


In [ ]:
sum(df_val['VL_qwen_phy_sum_grpo_full_lr2e5'] == '') ,  sum(df_val['VL_qwen_non_phy_sum_grpo_full_lr2e5'] == '')

(39, 176)

In [28]:
df_val.to_csv('val_VL_medical_qwen_draft_5rewards_grpo_only_drugADV_rougeBL.csv' , index = False)